In [1]:
import pandas as pd
import numpy as np

In [2]:
#Importar os datasets necessarios para a criacao desta tabela de factos
dffactos2=pd.DataFrame()
dfbase=pd.read_excel('Musica.xlsx')
dfdata=pd.read_csv('DimData.csv')
dflocal=pd.read_csv('DimLocalizacao.csv')
dfeurovisao=pd.read_csv('DimEurovisao.csv')
dfmusica=pd.read_csv('DimMusica_final.csv')
dfjunkconflito=pd.read_csv('DimJunkConflito.csv')
dfconflito=pd.read_excel('Conflitos.xlsx', sheet_name='Sheet2')
dfpop=pd.read_csv('Populacao.csv')
dfarea=pd.read_csv('area.csv')
dfgdp=pd.read_csv('gdp.csv')
dfturist=pd.read_csv('Turistas.csv')
dfco2=pd.read_csv('Emissoes.csv')

#Apagar as colunas do dataset base que nao sao relevantes para a tabela de factos
dfbase=dfbase.drop(['Value.#','Genre','Value.Language','EnglishNonEnglish',
                   'Value.Pl.','RunningOrder','Value.Host_Country','Value.Host_City',
                   'Value.Lyrics','Value.Lyrics translation','Ropercent','Value.#.1'],axis=1)
dfbase.rename(columns = {'Value.Sc.':'Pontuacao'}, inplace = True)

#Merge com os datasets que contem as medidas numericas na seguinte ordem:
#1-Merge, 2-Renomear coluna que vai ser mantida, 3-Apagar colunas nao necessarias
#Dataset Populacao
dfbase = pd.merge(dfbase, dfpop, how='left', left_on = ['Value.Country','Value.Year'] , right_on = ['Country','variable'])
dfbase.rename(columns = {'value':'Populacao'}, inplace = True)
dfbase=dfbase.drop(['Unnamed: 0','Country','variable'],axis=1)

#Dataset Area
dfbase = pd.merge(dfbase, dfarea, how='left', left_on = ['Value.Country','Value.Year'] , right_on = ['Country Name','variable'])
dfbase.rename(columns = {'value':'AreaKm2'}, inplace = True)
dfbase=dfbase.drop(['Unnamed: 0','Country Name','variable'],axis=1)

#Dataset PIB
dfbase = pd.merge(dfbase, dfgdp, how='left', left_on = ['Value.Country','Value.Year'] , right_on = ['Country Name','Year'])
dfbase=dfbase.drop(['Unnamed: 0','Country Name','Year','id'],axis=1)

#Dataset Turistas
dfbase = pd.merge(dfbase, dfturist, how='left', left_on = ['Value.Country','Value.Year'] , right_on = ['TIME','variable'])
dfbase=dfbase.drop(['Unnamed: 0','TIME','variable'],axis=1)
dfbase.rename(columns = {'value':'Turistas'}, inplace = True)

#Dataset Emissoes
dfbase = pd.merge(dfbase, dfco2, how='left', left_on = ['Value.Country','Value.Year'] , right_on = ['Country','variable'])
dfbase=dfbase.drop(['Unnamed: 0','Country','variable'],axis=1)
dfbase.rename(columns = {'value':'EmissoesCO2'}, inplace = True)

#Merge com os datasets que vao necessitar de criacao de chaves substitutas e eliminacao de colunas desnecessarias
#Merge com dimData
dfbase = pd.merge(dfbase, dfdata, how='outer', left_on = 'Value.Year', right_on = 'Ano')
dfbase=dfbase.drop(['MetadeDaDecada','Decada','Seculo','Ano'],axis=1)
#Merge com dimLocalizacao
dfbase = pd.merge(dfbase, dflocal, how='outer', left_on = 'Value.Country', right_on = 'Pais')
dfbase=dfbase.drop(['NumeroDeVizinhos','Continente','Regiao','Pais'],axis=1)
#Merge com dimEurovisao
dfbase = pd.merge(dfbase, dfeurovisao, how='outer', left_on = 'Value.Year', right_on = 'Ano')
dfbase=dfbase.drop(['RegraLinguagem','CidadeAnfitria','PaisAnfitriao','Ano','MaxPontuacaoPais','TotalPontosDisponivelConcurso',
                   'TotalPontosPaisDa', 'MaxPontosPaisADaPaisB','DescricaoPontos','VerificacaoTelevoto','VerificacaoVotoJuri',
                   'TotalParticipantes','NumeroEdicao'],axis=1)
#Merge com dimMusica
dfbase = pd.merge(dfbase, dfmusica, how='outer', left_on = ['Value.Song','Value.Artist'], right_on = ['Musica','Artista'])
dfbase=dfbase.drop(['Pontuacao_y','OrdemAtuacaoNormalizada','OrdemAtuacao','Classificacao','Lingua','Musica'],axis=1)

#Apagar algumas linhas a nulo que o programa criou

dfbase=dfbase.drop(dfbase.index[1644])
print(dfbase)

#Criacao da tabela de facto pela copia de colunas e operacoes entre os valores numericos
#A eurovisao de 2020 que nao foi realizada foi atribuido o codigo de 99 para poder converter a coluna para int
dffactos2['IDData']=dfbase['IDData'].astype(int)
dfbase['IDEurovisao'] = dfbase['IDEurovisao'].fillna(99)
dffactos2['IDEurovisao']=dfbase['IDEurovisao'].astype(int)
dffactos2['IDLocalizacao']=dfbase['IDLocalizacao'].astype(int)
dffactos2['IDMusica']=dfbase['IDMusica'].astype(int)
dffactos2['IDConflito']=dfbase['ConflitoJunk'].astype(int)
dfbase['Pontuacao_x'] = dfbase['Pontuacao_x'].fillna(0)
dffactos2['Pontos']=dfbase['Pontuacao_x'].astype(int)
dffactos2['AreaKm2']=dfbase['AreaKm2']
dffactos2['PIBPerCapita']=dfbase['PIBpercapita']
dffactos2['Populacao']=dfbase['Populacao']
dffactos2['DensidadePopulacional']=(dfbase['Populacao']/dfbase['AreaKm2'])
dffactos2['EmissoesCO2']=dfbase['EmissoesCO2']
dffactos2['EmissoesPorPopulacao']=(dfbase['EmissoesCO2']/dfbase['Populacao'])
dffactos2['EmissoesPorArea']=(dfbase['EmissoesCO2']/dfbase['AreaKm2'])
dffactos2['Turismo']=dfbase['Turistas']
dffactos2['TurismoPorArea']=(dfbase['Turistas']/dfbase['AreaKm2'])

#Exportar para CSV
dffactos2.to_csv("FactTable2_final.csv", index=False)

     Value.Country      Value.Artist  \
0      Netherlands       Jetty Paerl   
1      Netherlands     Corry Brokken   
2      Switzerland         Lys Assia   
3      Switzerland     Lys Assia (2)   
4          Belgium       Fud Leclerc   
...            ...               ...   
1639        Russia  Philipp Kirkorov   
1640      Slovenia     Darja Švajger   
1641         Spain      Anabel Conde   
1642        Turkey          Arzu Ece   
1643       Hungary     Csaba Szigeti   

                                             Value.Song  Pontuacao_x  \
0                                 De vogels van Holland          NaN   
1                                      Voorgoed voorbij          NaN   
2                                    Das alte Karussell          NaN   
3                                               Refrain          NaN   
4                       Messieurs les noyés de la Seine          NaN   
...                                                 ...          ...   
1639  Kolybel'n